In [1]:
import numpy as np 
from load_data import load_data
train, valid, test  = load_data('./mnist.pkl.gz')

D:\anaconda\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


... loading data


In [2]:
raw_X_train = train[0]
raw_y_train = train[1]
raw_X_val = valid[0]
raw_y_val = valid[1]
raw_X_test = test[0]
raw_y_test = test[1]

In [3]:
n_train_samples = raw_X_train.shape[0]
n_val_samples = raw_X_val.shape[0]
n_test_samples = raw_X_test.shape[0]

X_train = raw_X_train.reshape(n_train_samples, 28, 28, 1)
X_val = raw_X_val.reshape(n_val_samples, 28, 28, 1)
X_test = raw_X_test.reshape(n_test_samples, 28, 28, 1)

In [4]:
from keras import utils
Y_train = utils.to_categorical(raw_y_train, num_classes=10)
Y_val = utils.to_categorical(raw_y_val, num_classes=10)
Y_test = utils.to_categorical(raw_y_test, num_classes=10)

D:\anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras import Model
from keras import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l2
def mnist_model():
    X_input = Input(shape=(28, 28, 1), name='input')
    
    X = Conv2D(6, (5, 5), padding='same', kernel_initializer='he_normal', bias_initializer='zeros')(X_input)
    X =Activation('relu')(X)
    X= MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Conv2D(16,(5, 5), padding="same", kernel_initializer='he_normal', bias_initializer='zeros')(X)
    X = Activation("relu")(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Flatten()(X)
    X = Dense(120, kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(X)
    X = Activation("relu")(X)
    
    X = Dense(84, kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(X)
    X = Activation("relu")(X)
    
    X = Dense(10)(X)
    X_output = Activation("softmax")(X)
    
    model = Model(inputs = X_input, outputs = X_output)
    
    return model

In [6]:
reg_model = mnist_model()

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
from keras.optimizers import RMSprop
reg_model.compile(loss="categorical_crossentropy", optimizer=RMSprop(lr=0.001), metrics=["accuracy"])

In [9]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('reg_training.log')

In [10]:
history = reg_model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=128, epochs=20, verbose=1, callbacks=[csv_logger])

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 65s 1ms/step - loss: 1.5217 - acc: 0.8948 - val_loss: 0.5842 - val_acc: 0.9414
Epoch 2/20
50000/50000 [==============================] - 63s 1ms/step - loss: 0.3767 - acc: 0.9586 - val_loss: 0.3150 - val_acc: 0.9487
Epoch 3/20
50000/50000 [==============================] - 67s 1ms/step - loss: 0.2251 - acc: 0.9671 - val_loss: 0.1848 - val_acc: 0.9738
Epoch 4/20
50000/50000 [==============================] - 69s 1ms/step - loss: 0.1760 - acc: 0.9717 - val_loss: 0.1441 - val_acc: 0.9804
Epoch 5/20
50000/50000 [==============================] - 77s 2ms/step - loss: 0.1542 - acc: 0.9744 - val_loss: 0.1237 - val_acc: 0.9822
Epoch 6/20
50000/50000 [==============================] - 78s 2ms/step - loss: 0.1375 - acc: 0.9768 - val_loss: 0.1440 - val_acc: 0.9735
Epoch 7/20
50000/50000 [==============================] - 67s 1ms/step - loss: 0

In [11]:
(loss, accuracy) = reg_model.evaluate(X_test, Y_test, batch_size=128, verbose=1)
print("accuracy: {:.2f}%".format(accuracy * 100))

10000/10000 [==============================] - 5s 548us/step
accuracy: 98.56%


In [13]:
import pandas as pd
df = pd.read_csv('reg_training.log')
print(df)

    epoch      acc      loss  val_acc  val_loss
0       0  0.89476  1.521695   0.9414  0.584223
1       1  0.95858  0.376704   0.9487  0.315038
2       2  0.96706  0.225063   0.9738  0.184838
3       3  0.97166  0.176030   0.9804  0.144122
4       4  0.97438  0.154226   0.9822  0.123689
5       5  0.97678  0.137489   0.9735  0.143979
6       6  0.97822  0.125324   0.9804  0.115996
7       7  0.98000  0.116093   0.9743  0.132665
8       8  0.98112  0.110616   0.9821  0.108403
9       9  0.98138  0.106144   0.9834  0.098377
10     10  0.98154  0.101413   0.9859  0.092815
11     11  0.98294  0.096392   0.9769  0.113464
12     12  0.98312  0.093095   0.9852  0.090543
13     13  0.98334  0.089871   0.9838  0.090946
14     14  0.98420  0.087704   0.9874  0.081504
15     15  0.98432  0.085685   0.9842  0.087199
16     16  0.98510  0.083649   0.9806  0.094213
17     17  0.98518  0.079823   0.9840  0.089219
18     18  0.98598  0.077909   0.9839  0.090843
19     19  0.98582  0.077061   0.9860  0

In [12]:
def mnist_model_1():
    X_input = Input(shape=(28, 28, 1), name='input')
    
    X = Conv2D(6, (5, 5), padding='same', kernel_initializer='he_normal', bias_initializer='zeros')(X_input)
    X =Activation('relu')(X)
    X= MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Conv2D(16,(5, 5), padding="same", kernel_initializer='he_normal', bias_initializer='zeros')(X)
    X = Activation("relu")(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Flatten()(X)
    X = Dense(120, kernel_initializer='he_normal', kernel_regularizer=l2(0.01))(X)
    X = Activation("relu")(X)
    
    X = Dense(84, kernel_initializer='he_normal')(X)
    X = Activation("relu")(X)
    
    X = Dense(10)(X)
    X_output = Activation("softmax")(X)
    
    model = Model(inputs = X_input, outputs = X_output)
    
    return model

In [14]:
reg_model_1 = mnist_model_1()

In [15]:
from keras.optimizers import RMSprop
reg_model_1.compile(loss="categorical_crossentropy", optimizer=RMSprop(lr=0.001), metrics=["accuracy"])

In [16]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('reg_1_training.log')

In [17]:
history = reg_model_1.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=128, epochs=20, verbose=1, callbacks=[csv_logger])

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 55s 1ms/step - loss: 0.8321 - acc: 0.9065 - val_loss: 0.2270 - val_acc: 0.9721
Epoch 2/20
50000/50000 [==============================] - 69s 1ms/step - loss: 0.1879 - acc: 0.9672 - val_loss: 0.1442 - val_acc: 0.9750
Epoch 3/20
50000/50000 [==============================] - 83s 2ms/step - loss: 0.1346 - acc: 0.9746 - val_loss: 0.1153 - val_acc: 0.9795
Epoch 4/20
50000/50000 [==============================] - 96s 2ms/step - loss: 0.1160 - acc: 0.9775 - val_loss: 0.1448 - val_acc: 0.9718
Epoch 5/20
50000/50000 [==============================] - 68s 1ms/step - loss: 0.1070 - acc: 0.9789 - val_loss: 0.1178 - val_acc: 0.9769
Epoch 6/20
50000/50000 [==============================] - 61s 1ms/step - loss: 0.0980 - acc: 0.9807 - val_loss: 0.0881 - val_acc: 0.9848
Epoch 7/20
50000/50000 [==============================] - 82s 2ms/step - loss: 0.0921 - acc: 0.9829 - val_loss: 0.0988 - val_acc

In [19]:
(loss, accuracy) = reg_model_1.evaluate(X_test, Y_test, batch_size=128, verbose=1)
print("accuracy: {:.2f}%".format(accuracy * 100))

10000/10000 [==============================] - 6s 579us/step
accuracy: 98.79%


In [21]:
import pandas as pd
df = pd.read_csv('reg_1_training.log')
print(df)

    epoch      acc      loss  val_acc  val_loss
0       0  0.90648  0.832123   0.9721  0.227015
1       1  0.96720  0.187934   0.9750  0.144219
2       2  0.97464  0.134602   0.9795  0.115265
3       3  0.97754  0.116044   0.9718  0.144803
4       4  0.97888  0.106974   0.9769  0.117762
5       5  0.98074  0.098031   0.9848  0.088149
6       6  0.98292  0.092121   0.9808  0.098751
7       7  0.98400  0.085564   0.9820  0.091318
8       8  0.98410  0.081749   0.9847  0.080528
9       9  0.98516  0.076960   0.9865  0.075619
10     10  0.98534  0.075101   0.9834  0.083014
11     11  0.98636  0.070004   0.9833  0.081864
12     12  0.98704  0.068009   0.9861  0.074374
13     13  0.98760  0.065831   0.9872  0.073167
14     14  0.98808  0.063512   0.9860  0.074517
15     15  0.98802  0.060902   0.9832  0.078943
16     16  0.98848  0.059388   0.9879  0.063739
17     17  0.98840  0.058185   0.9886  0.061531
18     18  0.98908  0.055786   0.9877  0.065128
19     19  0.98892  0.055252   0.9894  0